# Train UnifiedQA

In [ ]:
!pip install transformers datasets sentencepiece

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir CausalQA
!cp -r /content/drive/MyDrive/CausalQA/input/* ./CausalQA/input/

In [ ]:
!unzip ./CausalQA/input/original-splits.zip
!unzip ./CausalQA/input/random-splits.zip

In [ ]:
import argparse
from argparse import Namespace
from datasets import load_dataset
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    DataCollatorForSeq2Seq,
    set_seed,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)

In [ ]:
# concatenate question+context with \\n as a separator
def build_input(batch):
    input_ = [
        (question + " \\n " + context if context is not None else question)
        for question, context in zip(
            batch["question_processed"], batch["context_processed"]
        )
    ]
    batch["input"] = input_
    return batch

In [ ]:
def generate_tokenizer(args):
  print("Load tokenizer and model...")
  tokenizer = T5Tokenizer.from_pretrained(args.checkpoint)
  model = T5ForConditionalGeneration.from_pretrained(args.checkpoint)

  return tokenizer, model

In [ ]:
print("Define tokenize_function_train...")
def tokenize_function_train(batches):
    encoded_inputs = tokenizer(
        batches["input"],
        max_length=args.source_length,
        padding="max_length",
        truncation=True,
    )
    encoded_answers = tokenizer(
        batches["answer"],
        max_length=args.target_length,
        padding="max_length",
        truncation=True,
    )
    encoded_inputs["labels"] = [
        [(a if a != tokenizer.pad_token_id else -100) for a in ans]
        for ans in encoded_answers["input_ids"]
    ]
    return encoded_inputs

Define tokenize_function_train...


In [ ]:
def train_unifiedqa(args):    

    print("Load dataset from csv file")
    train_dataset = load_dataset("csv", data_files=args.train_file)["train"]
    
    print("Map build_input")
    train_dataset = train_dataset.map(
        build_input, batched=True, load_from_cache_file=False, num_proc=args.num_procs
    )
    train_dataset = train_dataset.remove_columns(["context", "context_processed"])
    
    print("Map tokenize_function_train")
    train_dataset = train_dataset.map(
        tokenize_function_train,
        batched=True,
        load_from_cache_file=False,
        num_proc=args.num_procs,
    )
    train_dataset = train_dataset.remove_columns(["input", "answer"])

    print("DataCollator...")
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
    log_steps = args.steps // 10

    print("Set TrainingArguments...")
    train_args = Seq2SeqTrainingArguments(
        "models",
        per_device_train_batch_size=args.batch_size,
        max_steps=args.steps,
        seed=args.seed,
        save_strategy="no",
        logging_strategy="steps",
        logging_steps=log_steps,
        save_total_limit=1,
    )

    print("Seq2SeqTrainer...")
    trainer = Seq2SeqTrainer(
        model=model,
        args=train_args,
        train_dataset=train_dataset,
        data_collator=data_collator,
    )
    
    print("Train...")
    _ = trainer.train()

    start_index = args.train_file.rfind("/") + 1
    end_index = args.train_file.find("_")
    print("Save model...")
    trainer.save_model(args.output_directory + args.train_file[start_index:end_index])

In [ ]:
args = Namespace(
    checkpoint="allenai/unifiedqa-v2-t5-base-1363200",
    train_file="Webis-CausalQA-22-v-1.0/input/original-splits/squad2_train_original_split.csv",
    steps=6000,
    # source_length=2048, #original
    source_length=1024,
    target_length=100,
    batch_size=2,
    seed=42,
    num_procs=8,
    output_directory="Webis-CausalQA-22-v-1.0/models/original-splits/"
)

In [ ]:
set_seed(args.seed)

In [ ]:
tokenizer, model = generate_tokenizer(args)

Load tokenizer and model...


In [ ]:
train_unifiedqa(args)

In [ ]:
!cp -r Webis-CausalQA-22-v-1.0/models/original-splits/squad2 /content/drive/MyDrive/CausalQA/models/

# Load finetune model

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(args.checkpoint)

In [ ]:
finetuned_model = T5ForConditionalGeneration.from_pretrained('Webis-CausalQA-22-v-1.0/models/original-splits/squad2')

In [ ]:
args.train_file

'Webis-CausalQA-22-v-1.0/input/original-splits/squad2_train_original_split.csv'

In [ ]:
valid_dataset = load_dataset("csv", data_files="Webis-CausalQA-22-v-1.0/input/original-splits/squad2_valid_original_split.csv")

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
valid_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'question_processed', 'context', 'context_processed', 'answer', 'answer_processed'],
        num_rows: 252
    })
})

In [ ]:
valid_dataset = valid_dataset.map(
    build_input, batched=True, load_from_cache_file=False, num_proc=args.num_procs
)
valid_dataset = valid_dataset.remove_columns(["context", "context_processed"])

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#5:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

#6:   0%|          | 0/1 [00:00<?, ?ba/s]

#7:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
valid_dataset = valid_dataset.map(
        tokenize_function_train,
        batched=True,
        load_from_cache_file=False,
        num_proc=args.num_procs,
    )
valid_dataset = valid_dataset.remove_columns(["input", "answer"])

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

#6:   0%|          | 0/1 [00:00<?, ?ba/s]

#5:   0%|          | 0/1 [00:00<?, ?ba/s]

#7:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
valid_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'question_processed', 'answer_processed', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 252
    })
})

In [ ]:
sample = valid_dataset['train'][0]

In [ ]:
from pprint import pprint

In [ ]:
sample.keys()

dict_keys(['id', 'question', 'question_processed', 'answer_processed', 'input_ids', 'attention_mask', 'labels'])

In [ ]:
sample['id']

'56e1b00ce3433e140042309f'

In [ ]:
sample['question']

'What are two factors that directly effect how powerful a Turing machine may or may not be?'

In [ ]:
sample['question_processed']

'what are two factors that directly effect how powerful a turing machine may or may not be?'

In [ ]:
len(sample['input_ids'])

1024

In [ ]:
print(len(sample['attention_mask']))

1024


In [ ]:
print(sample['labels'])

[97, 42, 628, 97, 42, 628, 97, 42, 628, 1, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]


In [ ]:
tokenizer.decode([97, 42, 628, 97, 42, 628, 97, 42, 628, 1])

'time or space time or space time or space</s>'

In [ ]:
sample.keys()

dict_keys(['id', 'question', 'question_processed', 'answer_processed', 'input_ids', 'attention_mask', 'labels'])

In [ ]:
import torch

In [ ]:
sample_final = {}
for k, v in sample.items():
  if k in ['input_ids', 'attention_mask']:
    sample_final[k] = torch.LongTensor([v])

In [ ]:
sample_final.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
max_label = sample['labels'].index(1) + 1

In [ ]:
sample.keys()

dict_keys(['id', 'question', 'question_processed', 'answer_processed', 'input_ids', 'attention_mask', 'labels'])

In [ ]:
sam

In [ ]:
out = model.generate(**sample_final)
print("Question:", sample['question'])
print("Reference:", tokenizer.decode(sample['labels'][:max_label]))
print("Predict:", tokenizer.decode(out[0]))

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Question: What are two factors that directly effect how powerful a Turing machine may or may not be?
Reference: time or space time or space time or space</s>
Predict: <pad> time or space</s>


'<pad> time or space</s>'